Import di base

In [1]:
from gym import Env
from gym.spaces import Discrete, Box

from IPython.display import clear_output

import numpy as np
import random
import socket
import time

#import struct


In [34]:
# definizioni variabili globali ==================================================================
HOST = "127.0.0.1"
PORT = 7001
PACKET_SIZE = 1024

PacketCounter = 0
PacketCounter_lastCheck = 0

# definizioni funzioni =======================================================

#creo socket e lo faccio partire
def CreaSocket():
    # creo socket server per UE5
    MySocket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

    #Bindo il socket 
    MySocket.bind((HOST,PORT))

    #accendo il socket in ascolto
    MySocket.listen()

    return MySocket


#ciclo di attesa connesione
def AttesaConnesione(MySocket):
    while True:
        RemoteConnection , RemoteAddress = MySocket.accept()
        MySocket.close()
        break
    return RemoteConnection,RemoteAddress

#funzione per sampolare il robot di unreal
def MandaRicevi_Robot(MySocket, R_Start=0.0, R_Arm_1=0.0, R_Arm_2=0.0, R_Arm_3=0.0, R_Arm_4=0.0, R_End=0.0):
    MySocket.sendall(bytes(
                    str(R_Start) +"|"+ 
                    str(R_Arm_1) +"|"+ 
                    str(R_Arm_2) +"|"+ 
                    str(R_Arm_3) +"|"+ 
                    str(R_Arm_4) +"|"+ 
                    str(R_End)
                    ,"utf-8"))
    
    ReceivedData = MySocket.recv(PACKET_SIZE)
    ReceivedData = ReceivedData.decode("utf-8")
    return float(ReceivedData)

# classe environment di gioco =============================================

class RobotEnv(Env):
    def __init__(self):
        # azioni che possiamo fare
        self.action_space = Discrete(4)
        # Array distanze dal target
        self.observation_space = Box(low=np.array([0]),high=np.array([120]))
        # distanza di partenza
        self.state = 0.0

        self.angoloAttuale_1 = random.uniform(0,90)
        self.angoloAttuale_2 = random.uniform(0,90)

        # lunghezza esercizio
        self.Exercise_length = 60

    def step(self, action,verbosita=False):
        global UE5_Socket
        
        # applichiamo azione
        
        if action == 0:
            self.angoloAttuale_1 += 1
        elif action == 1:
            self.angoloAttuale_1 -= 1
        elif action == 2:
            self.angoloAttuale_2 += 1
        elif action == 3:
            self.angoloAttuale_2 -= 1

        self.state = MandaRicevi_Robot(UE5_Socket,-90.0,self.angoloAttuale_1,self.angoloAttuale_2)

        # riduzione tempo doccia
        self.Exercise_length -= 1

        #calcolo reward
        if self.state <= 40:
            reward = 1
        else:
            reward = -3 * (self.state / 100)

        #verfica se doccia finita
        if self.Exercise_length <= 0:
            done = True
        else:
            done = False
        
        # segnaposto per info
        info = {}
        
        # ritorniamo informazioin sullo step
        return self.state, reward, done, info

    def render(self,mode=0):
        print("Tempo: "+str(self.Exercise_length) + "-- distanza: " + str(self.state))

    def reset(self):
        #resettiamo temperatura
        self.angoloAttuale_1 = random.uniform(0,90)
        self.angoloAttuale_2 = random.uniform(0,90)
        self.state = 0.0
        # resettiamo tempo
        self.Exercise_length = 60
        return self.state

# codice ==================================================================

#creo e ascolto
Server_Socket = CreaSocket()

#attendo e connetto con UE5 
UE5_Socket , UE5_Address = AttesaConnesione(Server_Socket)

#confermo connesione aperta
print(f"Aperto socket con UE5: {UE5_Address}")


SimpleRobotEnv = RobotEnv()


Aperto socket con UE5: ('127.0.0.1', 51330)


c:\Users\xmari\.conda\envs\aaaa\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [39]:
# giro di prova dell'exercise
episodes = 1
for episode in range(1, episodes+1):
    state = SimpleRobotEnv.reset()
    done = False
    score = 0 
    
    while not done:
        SimpleRobotEnv.render(mode=0)
        action = SimpleRobotEnv.action_space.sample()
        n_state, reward, done, info = SimpleRobotEnv.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Tempo: 60-- distanza: 47.744232
Tempo: 59-- distanza: 74.062721
Tempo: 58-- distanza: 74.062721
Tempo: 57-- distanza: 72.136467
Tempo: 56-- distanza: 74.062721
Tempo: 55-- distanza: 72.136467
Tempo: 54-- distanza: 72.136467
Tempo: 53-- distanza: 70.216499
Tempo: 52-- distanza: 72.136467
Tempo: 51-- distanza: 72.136467
Tempo: 50-- distanza: 70.216499
Tempo: 49-- distanza: 70.216499
Tempo: 48-- distanza: 68.303955
Tempo: 47-- distanza: 70.216499
Tempo: 46-- distanza: 68.303955
Tempo: 45-- distanza: 70.216499
Tempo: 44-- distanza: 68.303955
Tempo: 43-- distanza: 68.303955
Tempo: 42-- distanza: 70.216499
Tempo: 41-- distanza: 72.136467
Tempo: 40-- distanza: 74.062721
Tempo: 39-- distanza: 75.994225
Tempo: 38-- distanza: 75.994225
Tempo: 37-- distanza: 74.062721
Tempo: 36-- distanza: 72.136467
Tempo: 35-- distanza: 70.216499
Tempo: 34-- distanza: 68.303955
Tempo: 33-- distanza: 68.303955
Tempo: 32-- distanza: 66.400101
Tempo: 31-- distanza: 66.400101
Tempo: 30-- distanza: 64.506287
Tempo: 2

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.optimizers import Adam

states = SimpleRobotEnv.observation_space.shape
actions = SimpleRobotEnv.action_space.n

def build_model(states,actions):
    model = Sequential()
    model.add(Dense(256,activation="relu",input_shape=states))
    model.add(Dense(256,activation="relu"))
    model.add(Dense(actions,activation="linear"))
    return model

In [42]:
del model # da usare quando da errore sotto

In [43]:
model = build_model(states,actions)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 256)               512       
_________________________________________________________________
dense_10 (Dense)             (None, 256)               65792     
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 771       
Total params: 67,075
Trainable params: 67,075
Non-trainable params: 0
_________________________________________________________________


In [44]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory

def build_agent(model, actions):
    policy = BoltzmannGumbelQPolicy()
    memory = SequentialMemory(limit=50000,window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

In [49]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3),metrics=["mae"])
dqn.fit(SimpleRobotEnv, nb_steps=50000,visualize=False,verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 179s 18ms/step - reward: -1.6802
166 episodes - episode_reward: -101.166 [-238.763, 60.000] - loss: 41.347 - mae: 34.134 - mean_q: -43.070

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 181s 18ms/step - reward: -1.3637
167 episodes - episode_reward: -81.378 [-198.164, 57.779] - loss: 40.210 - mae: 31.271 - mean_q: -40.431

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 182s 18ms/step - reward: -1.4175
167 episodes - episode_reward: -85.209 [-184.938, 60.000] - loss: 40.035 - mae: 31.631 - mean_q: -41.692

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 182s 18ms/step - reward: -1.5534
166 episodes - episode_reward: -93.567 [-277.766, 60.000] - loss: 41.437 - mae: 31.839 - mean_q: -42.525

Interval 5 (40000 steps performed)
 7430/10000 [=====================>........] - ETA: 47s

In [56]:
scores = dqn.test(SimpleRobotEnv, nb_episodes=10,visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 10 episodes ...
Episode 1: reward: -124.720, steps: 60
Episode 2: reward: -148.896, steps: 60
Episode 3: reward: -83.634, steps: 60
Episode 4: reward: -78.488, steps: 60
Episode 5: reward: -139.435, steps: 60
Episode 6: reward: 60.000, steps: 60
Episode 7: reward: -157.614, steps: 60
Episode 8: reward: -127.287, steps: 60
Episode 9: reward: -129.329, steps: 60
Episode 10: reward: 60.000, steps: 60
-86.94025008300002


Chiusure varie

In [ ]:
# spengo socket prima di uscire
UE5_Socket.shutdown(socket.SHUT_RDWR)
UE5_Socket.close()

Codice vario utile:

In [ ]:
'''
DistanzaAttuale = MandaRicevi_Robot(
                            UE5_Socket,
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0),
                            random.uniform(-230.0,230.0)
                            )
'''

'\nDistanzaAttuale = MandaRicevi_Robot(\n                            UE5_Socket,\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0),\n                            random.uniform(-230.0,230.0)\n                            )\n'